In [88]:
import neuroglancer
import numpy as np
import math

Create a new (initially empty) viewer.  This starts a webserver in a background thread, which serves a copy of the Neuroglancer client, and which also can serve local volume data and handles sending and receiving Neuroglancer state updates.

In [89]:
viewer = neuroglancer.Viewer()

Print a link to the viewer (only valid while the notebook kernel is running). Note that while the Viewer is running, anyone with the link can obtain any authentication credentials that the neuroglancer Python module obtains. Therefore, be very careful about sharing the link, and keep in mind that sharing the notebook will likely also share viewer links.

In [90]:
viewer

http://127.0.0.1:32869/v/b951f6874409c7a75c6d048d9c4e2749ce86e606/

Add some example layers using the precomputed data source (HHMI Janelia FlyEM FIB-25 dataset).

In [92]:
with viewer.txn() as s:
  s.layers['image'] = neuroglancer.ImageLayer(source='precomputed://https://bossdb-open-data.s3.amazonaws.com/iarpa_microns/minnie/minnie65/em')
  s.layers['segmentation'] = neuroglancer.SegmentationLayer(source='precomputed://https://storage.googleapis.com/iarpa_microns/minnie/minnie65/seg_m943/', selected_alpha=0.3) # graphene://middleauth+https://minnie.microns-daf.com/segmentation/table/minnie65_public 
  # skeleton layer only works in the Allen Institute network
  s.layers['skeleton'] = neuroglancer.SegmentationLayer(source='precomputed://http://bigkahuna.corp.alleninstitute.org/ACdata/Users/wanqing/Neuroglancer/skel_axon/skeletons/') 
  s.dimensions = neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=[4, 4, 40])


In [93]:
rid = 864691136903144370
sid = 236197
cell_type = 'L6short-a' 		
soma_pos = [152681, 226226, 19189]

In [94]:
# make a cetain segment visible
with viewer.txn() as s:
  s.layers['segmentation'].segments = [rid]
  s.layers['skeleton'].segments = [rid]
  s.voxel_coordinates = soma_pos

In [7]:
# set up path for skeletons

import skeleton_plot as skelplot
import skeleton_plot.skel_io as skel_io

#raw skeleton files
skel_path = 'https://storage.googleapis.com/allen-minnie-phase3/minniephase3-emily-pcg-skeletons/minnie_all/v661/skeletons/'

skel_dir = str(rid)+'_'+str(sid) 
skel_filename = str(rid)+'_'+str(sid)+'.swc'

# raw skel
skel = skel_io.read_skeleton(skel_path, skel_filename)
# get the axon segment out
axon_indices = np.where(skel.vertex_properties['compartment'] == 2)[0]

/home/wanqing.yu/anaconda3/envs/EM/lib/python3.11/site-packages/python_jsonschema_objects/__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")


In [61]:
import navis
skel_navis = navis.read_swc(skel_path+skel_filename)
skel_seg1 = navis.TreeNeuron(skel_navis.nodes[skel_navis.nodes.node_id.isin(skel_navis.segments[0])])
skel_seg1.nodes['x'], skel_seg1.nodes['y'], skel_seg1.nodes['z'] = skel_seg1.nodes['x'] * 1000 / 4, skel_seg1.nodes['y'] * 1000 / 4, skel_seg1.nodes['z'] * 1000 / 40
skel_seg1_sm = navis.smooth_skeleton(skel_seg1, window=10, to_smooth=['x', 'y', 'z'])

/home/wanqing.yu/anaconda3/envs/EM/lib/python3.11/site-packages/navis/graph/graph_utils.py:399: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.nodes['type'] = 'slab'
/home/wanqing.yu/anaconda3/envs/EM/lib/python3.11/site-packages/navis/graph/graph_utils.py:412: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.nodes['type'] = x.nodes['type'].astype(cat_types)
/tmp/ipykernel_3005360/3589417375.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

Smoothing:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
import copy
import time

def interpolate_to(final_state, frames_per_second=5, seconds=1):
    total_frames = int(round(seconds * frames_per_second))
    initial_state = viewer.state
    for frame_i in range(total_frames):
        t = frame_i / total_frames
        viewer.set_state(
            neuroglancer.ViewerState.interpolate(initial_state, final_state, t)
        )
        time.sleep(1 / frames_per_second)
    viewer.set_state(final_state)

def move_to(final_voxel_coordinates, final_orientation, **kwargs):
    final_state = copy.deepcopy(viewer.state)
    final_state.voxel_coordinates = final_voxel_coordinates
    final_state.crossSectionOrientation = final_orientation
    final_state.projectionOrientation = final_orientation
    interpolate_to(final_state, **kwargs)

def do_move_to(final_voxel_coordinates, final_orientation, seconds=1, frames_per_second=10):
    move_to(final_voxel_coordinates, final_orientation = final_orientation, seconds=seconds, frames_per_second=frames_per_second)


In [11]:
# set the initial state
with viewer.txn() as s:
    s.voxel_coordinates = [skel_seg1.nodes['x'][0], skel_seg1.nodes['y'][0], skel_seg1.nodes['z'][0]]

In [81]:
# get the dotprops
skel_dp = navis.make_dotprops(skel_seg1, k=20)


In [73]:
def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm > 0 else v

def surface_normal_to_quaternion(normal):
    normal = normalize(normal)
    up_vector = np.array([0, 0, 1])
    axis = np.cross(up_vector, normal)
    axis = normalize(axis)
    angle = np.arccos(np.dot(up_vector, normal))
    
    qx = axis[0] * np.sin(angle / 2)
    qy = axis[1] * np.sin(angle / 2)
    qz = axis[2] * np.sin(angle / 2)
    qw = np.cos(angle / 2)
    
    return [qx, qy, qz, qw]

def angle_between_vectors(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    cos_angle = dot_product / (magnitude_v1 * magnitude_v2)
    cos_angle = np.clip(cos_angle, -1, 1)  # Ensure within valid range for arccos
    angle_radians = np.arccos(cos_angle)
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

def flip_vectors_if_necessary(skel_dp_vect):
    """Flip the direction of vectors if the angle with the previous vector is > 140 degrees."""
    for i in range(1, len(skel_dp_vect)):
        angle = angle_between_vectors(skel_dp_vect[i], skel_dp_vect[i-1])
        if angle > 140:
            skel_dp_vect[i] = abs(skel_dp_vect[i]) * np.sign(skel_dp_vect[i-1])
    return skel_dp_vect


In [38]:
skel_dp.vect = flip_vectors_if_necessary(skel_dp.vect)

In [95]:
# get the quaternion for each node
quaternions = []
for i in range(len(skel_dp)):
    quaternions.append(surface_normal_to_quaternion(skel_dp.vect[i]))
quaternions

[[0.6936608958321225, -0.19407541102575573, 0.0, 0.6936636767400763],
 [0.6936608958321225, -0.19407541102575573, 0.0, 0.6936636767400763],
 [0.6936608958321225, -0.19407541102575573, 0.0, 0.6936636767400763],
 [0.6936608958321225, -0.19407541102575573, 0.0, 0.6936636767400763],
 [0.6936608958321225, -0.19407541102575573, 0.0, 0.6936636767400763],
 [0.6936608915033204, -0.1940754312964487, 0.0, 0.6936636753974624],
 [0.6936608915033204, -0.1940754312964487, 0.0, 0.6936636753974624],
 [0.6936608958321225, -0.19407541102575573, 0.0, 0.6936636767400763],
 [0.6727832188559818, -0.23977711100957957, 0.0, 0.6999069062823168],
 [0.6727832188559818, -0.23977711100957957, 0.0, 0.6999069062823168],
 [0.6727832194701172, -0.23977711122845483, 0.0, 0.699906905616998],
 [0.6560478661968485, -0.26778712776218144, 0.0, 0.7056140952839896],
 [0.6494388515444586, -0.2576383439358814, 0.0, 0.7154380908496494],
 [0.6524661196776411, -0.23425647309808054, 0.0, 0.7207023431933952],
 [0.6468925495323387, -0

In [96]:
# set the initial state
with viewer.txn() as s:
    s.voxel_coordinates = [skel_seg1.nodes['x'][0], skel_seg1.nodes['y'][0], skel_seg1.nodes['z'][0]]
    s.crossSectionOrientation = quaternions[0]

In [97]:
# do move to according to the list of positions and orientations in df
for i in range(len(skel_seg1.nodes)): 
    do_move_to([skel_seg1.nodes['x'][i], skel_seg1.nodes['y'][i], skel_seg1.nodes['z'][i]], final_orientation = quaternions[i], seconds = 2)

Stop the Neuroglancer web server, which invalidates any existing links to the Python-tied viewer.

In [87]:
neuroglancer.stop()